In [10]:
%run imports
from funciones import *
from Recursos.monitor_resources import *

print_gpu_usage()
print_ram_usage()

=== GPU 0 (NVIDIA GeForce GTX 1660) ===
  Load: 4.0%
  Free Memory: 5.09 GB
  Used Memory: 0.74 GB
  Total Memory: 6.00 GB
  Temperature: 43.0 °C
=== RAM Usage ===
Total RAM: 15.95 GB
Available RAM: 6.31 GB
Used RAM: 9.63 GB
RAM Usage Percentage: 60.4%


In [11]:
df_valid = pd.read_csv("output/valid_photos.csv")
df_valid.head()

,photo_id,business_id,caption,label
0,zsvj7vloL4L5jhYyPIuVwg,Nk-SJhPlDBkAZvfsADtccA,Nice rock artwork everywhere and craploads of ...,inside
1,HCUdRJHHm_e0OCTlZetGLg,yVZtL5MmrpiivyCIrVkGgA,NaN,outside
2,vkr8T0scuJmGVvN2HJelEA,_ab50qdWOk0DdB6XOrBitw,oyster shooter,drink
3,pve7D6NUrafHW3EAORubyw,SZU9c8V2GuREDN5KgyHFJw,Shrimp scampi,food
4,H52Er-uBg6rNrHcReWTD2w,Gzur0f0XMkrVxIwYJvOt2g,NaN,food


In [19]:
df_reduced = dim_reduction(df_valid, 10)
df_reduced["label"].value_counts()

El tamaño del nuevo DataFrame es: 19000


label
food       10267
inside      5325
outside     1765
drink       1484
menu         159
Name: count, dtype: int64

In [20]:
# Definimos la secuencia de aumentación
df_homogenized = homogenize_dataset(df_reduced, 2000) 
df_homogenized['label'].value_counts()

Processing Categories: 5it [00:26,  5.28s/it]


label
inside     5325
outside    2000
drink      2000
menu       2000
food       2000
Name: count, dtype: int64

In [21]:
df_homogenized.to_csv("output/df_homogenized.csv", index=False) # creacion de csv 